## 3.1 Gene counts based expression analysis

### I) What are the components of a FASTQ file?

__Line1: __Sequence identifier (start with @)  
__Line2: __Sequence (usually nucleotide sequence: A/T/C/G/N)  
__Line3: __Quality score identifier line (start with "+"; or only consisting a "+")  
__Line4: __Quality score (for the corresponding sequence in Line 2)

__REFERENCE__
{1} FASTQ Files https://help.basespace.illumina.com/articles/descriptive/fastq-files/
{2} FASTQ format - Wikipedia https://www.wikiwand.com/en/FASTQ_format

### II) What is a SAM file?

SAM stands for __Sequence Alignment/Map format__. It is a TAB-delimited text format consisting of an optional header section (start with @), and followed by an alignment section. Each alignment line has 11 fields for: Query template name, bitwise flag, reference sequence name, 1-based leftmost mapping position, mapping quality, CIGAR string, reference name of the mate/next read, position of the mate/next read, observed templated length, segment sequence and ASCII of phred-scaled based quality.

__Reference__  
{1} SAM Format Specification, 18 Oct 2018, https://samtools.github.io/hts-specs/SAMv1.pdf

### III) Why is it not possible to run htseq-count directly on FASTQ files? What is the missing step?

FASTQ files are designed to store raw data of biological sequence and corresponding qulity scores from the sequencer, while not including the enough required information (i.e. CIGAR) to run `htseq-count`. `htseq-count` is to count the number of reads mapping to each feature, given by aligned sequencing reads and a list of genomic features. To fill in the gap, we need to bridge these steps by mapping the FASTQ files to reference genome (i.e. HTSAT2) and convert to SAM format for `htseq-count`.

__Reference__  
{1} HTSeq 0.8.0 documentation, https://htseq.readthedocs.io/en/master/count.html

### IV) Find one example of an optional SAM tag and explain why it might be useful.

Optional SAM tag might not present in general, but some specific software or package would require those field as necessary input. In SAM format specification document, the authors indicated that __TC__ tag (the number of segments in the template) should be present if the template has more than 2 segemnts.

Other examples are broadly used by bowtie2, which would record the type of alignment, only present in specific condition (i.e. if SAM record is for an aligned read, if the SAM record is for a read that aligned as part of a paired-end alignment or for reads that were filtered out.)

__REFERENCE__  
{1} SAM Format Specification, 18 Oct 2018, https://samtools.github.io/hts-specs/SAMv1.pdf  
{2} Optional Fields Specification, 26 Jul 2018, https://samtools.github.io/hts-specs/SAMtags.pdf  
{3} bowtie2 manual http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml

### V) You practiced the steps from FASTQ to BAM but is it possible to go from BAM to FASTQ? If so, how?

There's several way possible to revert BAM files to FASTQ format. 

Most people recommend __Bedtools__{1}:
>`bedtools bamtofastq -i input.bam -fq output_reverted.fastq`
>- paired-end reads:  
`samtools sort -n input.bam -o input_sorted.bam 
bedtools bamtofastq -i input_sorted.bam -fq output_r1.fastq -fq2 output_r2.fastq`

or __Samtools__{1}:
>`samtools bam2fq intput.bam > output_reverted.fastq`  
>- paired-end reads:  '/1' or '/2' is added to the end of read names


Other tools like __Picard__{1}:
>`java -Xmx2g -jar Picard/SamToFastq.jar I=intput.bam F=output_r1.fastq F2=output_r2.fastq`
 - Add F2 to get two files for paired-end reads (R1 and R2).
 
or __HTSlib__ in __GATK__{2}:
>`htscmd bamshuf -uOn 128 aln_reads.bam tmp > shuffled_reads.bam 
htscmd bam2fq -a shuffled_reads.bam > interleaved_reads.fastq`


__REFERENCE__:  
{1} Metagenomics - Converting BAM to fastq http://www.metagenomics.wiki/tools/samtools/converting-bam-to-fastq  
{2} Revert a BAM file to FastQ format https://gatkforums.broadinstitute.org/gatk/discussion/2908/howto-revert-a-bam-file-to-fastq-format


### VI) Describe the process of a spliced alignment and explain why it is more difficult than mapping genomic DNA reads.

Spliced alignment need to perform when we have a RNA sequencing read from more then two connected exons. Fragments of this read would sequencially map to multiple downstream location separated by thousands base pair away. Program would need to test for the possible junction of exon inside the reads, identified by GT-AG dinucleotides.

__REFERENCE__  
{1} Spliced alignment (Bioinformatics) http://what-when-how.com/bioinformatics/spliced-alignment-bioinformatics/

### VII) Using the default settings of htseq-count, would a read which mapped fully to a genomic region on which 2 genes overlap be counted towards any of the 2 genes? Explain why.

By default, `htseq-count` will consider this situation to be "ambiguous", which would discard the same fraction of reads in all samples. For the purpose to compare the relative difference of expression level, to discard such reads would underestimate the absolute count but keep fold-change correct. Otherwise, counting these reads for both genes might casue false-positive counts when only one of the gene change the expression level, the results would be bias and effect both of the 2 genes.

__Reference__  
{1} HTSeq 0.8.0 documentation, https://htseq.readthedocs.io/en/master/count.html

### VIII) You are interested in RNA-sequencing an organism for which no transcriptome annotations are available. Can you still use htseq-count to examine gene expression? If so, what upstream analyses would you perform first?

It's possible to do so by creating customized annotation from the closest well annotated species. First use `bedtools genomecov` to extract the position of all transcribed sequences in the sample, and then use `fastaFromBed` to get sequences from the reference genome and `blastx` to find the best-matched annotated species. Map with this customized annotation reference would be able to make BAM files as input for the downstream calculation by `htseq-count`.


__REFERENCE__  
{1} Biostars - Rna Seq On Unannotated Genome https://www.biostars.org/p/75462/  
{2} Biostars - Rna-Seq With Reference Genome But No Gene Annotations https://www.biostars.org/p/75323/

### IX) Intron retention can result in mRNA isoforms that contain unspliced introns. If **sample 1** contains a high frequency of intron-retained isoforms of gene A whereas **sample 2** contains fully spliced isoforms of gene A, but each sample has the identical number of reads mapping to gene A, which sample will have a higher gene A count using default htseq-count settings?

The result would be the same becuase `htseq-count` counts exons only.

__REFERENCE__  
{1} Biostars - htseq counts on genes (introns + exons) https://www.biostars.org/p/221422/  
{2} Biostars - shall intronic reads be considered in RNAseq count? https://www.biostars.org/p/119598/